<a href="https://colab.research.google.com/github/Carlosvergara1995/Problem_Set_4_Predicting_Tweets/blob/main/Analizador_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##TProblem_Set_4_Predicting_Tweets


##CARGA DE DATOS

In [1]:
#IMPORTAMOS LIBRERIAS
import pandas as pd
import numpy as np
import pandas as pd

In [2]:
!pip install nltk
!python -m nltk.downloader stopwords
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 4.1 MB/s eta 0:00:00


# Carga de datos

In [3]:
###Cargamos datos
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

df_test.shape


FileNotFoundError: ignored

In [ ]:
print(df_train.shape)
#Calculamos na
print("Los NA son:")
print(df_train.isna().sum())